In [6]:
import torch, torchvision
import torchvision.transforms.functional as fn
torch.manual_seed(0)


import os
import numpy as np
import matplotlib.pyplot as plt

In [7]:
#Update os
path = "/home/kishen/documents/python_projects/dogs_v_cats/"
os.chdir(path)
from src.dataset import ImageDataset


Preprocessing

In [22]:
#Create labels: cats=0, dogs=1
labels = np.zeros(25_000)
labels[12_500:] =  1


In [24]:
type(labels)

numpy.ndarray

In [10]:
#create augmentations for ImageDataset
path = "/home/kishen/documents/python_projects/dogs_v_cats/"
os.chdir(path)
from src.dataset import ImageDataset

transformations = torch.nn.Sequential(
        torchvision.transforms.Resize((300,300)),
        torchvision.transforms.RandomPerspective(p=.25),
        torchvision.transforms.RandomHorizontalFlip(p=.25)

)

#Build training dataset
training_data = ImageDataset(image_paths= "/home/kishen/documents/python_projects/dogs_v_cats/data/train/",
                            targets=labels,
                            augmentations= transformations)



In [11]:
from torch.utils.data import DataLoader

loader_params = {"batch_size": 1,
                 "shuffle": True,
                 "num_workers": 16}
train_loader = DataLoader(training_data, **loader_params)

**Training**

In [12]:
os.getcwd()

'/home/kishen/documents/python_projects/dogs_v_cats'

In [13]:
#set hyperparameters
epochs = 3
criterion = torch.nn.BCELoss()
model_params={'lr':.0001,
             'momentum':.9}

#init model and optimizer
from src.models import ConvNet

model = ConvNet()
optimizer = torch.optim.SGD(model.parameters(), **model_params)

#move model to gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

cuda


In [20]:
for epoch in range(epochs):
    running_loss = 0

    for i, data in enumerate(train_loader):
        
        #move data to gpu
        inputs, labels = data['image'], data['target']
        inputs, labels = inputs.to(device), labels.float().to(device)

        #zero gradients
        optimizer.zero_grad() 

        #model computations
        # labels = labels.unsqueeze(1)
        # labels = labels.float()
        preds = model(inputs.type(torch.cuda.FloatTensor))
        loss = criterion(preds, labels)

        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print("running loss: ", running_loss)



ValueError: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 2])) is deprecated. Please ensure they have the same size.